# Концепт

Я решила взять максимально гомогенные данные, то есть, вместо абсолютно случайных отзывов на фильмы я взяла один и тот же жанр; пойдя дальше, все фильмы, что я взяла, так или иначе относятся к Звёздным Войнам.

Источник данных - rottentomatoes, потому что там есть бинарная оценка отзывов (rotten-fresh); с типичными вещами вроде 5-звёздочной и 10-звёздочной системы оценивания очень много основывалось бы на субъективном делении на хорошее и плохое (3 звезды? 5 звёзд?).

В каждом отзыве на rottentomatoes есть ссылка на полный отзыв, но источники там совершенно разные и универсального парсера придумать не получилось, поэтому я довольствовалась короткими отзывами в одно-три предложения. К тому же, кажется, что такие маленькие отзывы легче классифицировать. 

# Импорт библиотек

In [ ]:
!pip install bs4

In [ ]:
!pip install fake_useragent

In [237]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import nltk
from collections import Counter
import pandas as pd
import random
from sklearn.metrics import accuracy_score

In [238]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Парсинг

In [239]:
ua = UserAgent(verify_ssl = False)
headers = {"UserAgent": ua.random}
session = requests.session()

In [240]:
positive_reviews = []
negative_reviews = []

In [241]:
def fetch_film_reviews(url):
    response_fresh = session.get(url+"?sort=fresh", headers=headers)
    soup =  BeautifulSoup(response_fresh.text)
    fresh_reviews = [tag.get_text() for tag in soup.find_all(attrs={"data-qa": "review-quote"})]
    positive_reviews.extend(fresh_reviews)
    response_rotten = session.get(url+"?sort=rotten", headers=headers)
    soup =  BeautifulSoup(response_rotten.text)
    rotten_reviews = [tag.get_text() for tag in soup.find_all(attrs={"data-qa": "review-quote"})]
    negative_reviews.extend(rotten_reviews)
    return positive_reviews, negative_reviews

In [242]:
films = ["https://www.rottentomatoes.com/m/star_wars_the_rise_of_skywalker/reviews",
"https://www.rottentomatoes.com/m/star_wars_the_last_jedi/reviews",
"https://www.rottentomatoes.com/m/rogue_one_a_star_wars_story/reviews",
"https://www.rottentomatoes.com/m/clone_wars/reviews"]

In [243]:
for film in films:
    positive, negative = fetch_film_reviews(film)

Всего я взяла 100 положительных и 100 отрицательных отзывов, дальше ещё возьму 20 тестовых. 

In [244]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(['star', 'wars', 'last', 'jedi', 'rise', 'skywalker', 'rogue', 'clone', 'one', 'full', 'review', 'spanish'])

Я решила исключить из частотных слов все, связанные с названиями фильмов, а ещё части фразы "Full review in Spanish". На момент парсинга и очистки данных мне не показалось, что она сильно релевантна, но когда я чуть позже увидела частотные словари, я поняла, что ошибалась. ~~Будем считать, что тот факт, что оригинальный отзыв на испанском, не вносит вклад в его положительность или отрицательность.~~

# Токенизация, лемматизация и составление множеств тональных слов

1. Функция для составления частотных словарей. В неё подаётся список с отзывами, каждый отзыв отдельно токенизируется, приводится в нижний регистр, каждый токен лемматизируется и проверяется на то, является ли он словом.
2. Два множества слов, полученные с помощью этой функции.
3. Ищем пересечение множеств, убираем его из обоих множеств так, что в них остаются только tone-specific слова (ну или почти...)

Важно: я (почти) не фильтровала по частотности, потому что осознала, что довольно большая часть тональных слов, к которым хотелось бы, чтобы определяющая функция была чувствительна, обладает примерно частотностью n=2. Обрезав длину словаря в самой функции (max_len = 100), я приблизительно избавилась от хвоста из слов с n=1.

In [245]:
def fetch_freqlist(reviews_list, max_len = 100):
    freqlist = Counter()
    for sentence in reviews_list:
        for word in nltk.word_tokenize(sentence.lower()):
            if word.isalpha() and word not in stopwords:
                freqlist[lemmatizer.lemmatize(word)] += 1
    return dict(freqlist.most_common(max_len))

In [246]:
positive_uncleaned = set(fetch_freqlist(positive).keys())
negative_uncleaned = set(fetch_freqlist(negative).keys())

In [247]:
intersection = set.intersection(positive_uncleaned, negative_uncleaned)
positive_dict = positive_uncleaned - intersection
negative_dict = negative_uncleaned - intersection

# Определение тона

Я предположила, что если количество положительных и отрицательных маркеров одинаково, то это негативный отзыв. К сожалению, нейтральную категорию мы ввести не можем (в целом, у нас её и в исходных данных тоже нет). 

In [248]:
def tone_detect(review_text):
    tone_dict = {'positive': 0, 'negative': 0}
    for word in nltk.word_tokenize(review_text):
        if lemmatizer.lemmatize(word) in positive_dict:
            tone_dict['positive'] += 1
        elif word in negative_dict:
            tone_dict['negative'] += 1
    if tone_dict['positive'] > tone_dict['negative']:
        return 1
    else:
        return 0

In [249]:
positive_test, negative_test = fetch_film_reviews("https://www.rottentomatoes.com/m/star_wars_episode_vii_the_force_awakens/reviews")
positive_test = positive_test[:10]
negative_test = negative_test[:10]

Тут я решила разметить позитивные и негативные отзывы, рандомно перемешать их.

А также разложить разметку и отзывы по разным спискам, чтобы можно было прогнать отзывы через функцию, и сравнить настоящую разметку с предсказаниями.

In [250]:
df = pd.DataFrame({'review': positive_test + negative_test, 
                   'tone': [1]*len(positive_test) + [0]*len(negative_test)})

df = df.sample(frac=1).reset_index(drop=True)

reviews = df['review'].tolist()
gold_tone = df['tone'].tolist()

In [251]:
predictions = [] 
for review in reviews: 
    predictions.append(tone_detect(review))

In [ ]:
gold_tone

In [ ]:
predictions

In [252]:
accuracy_score(predictions, gold_tone)

0.85

И тут мне поплохело, потому что я перезапускала ноутбук несколько раз, и 0.85-0.95 accuracy на невероятно наивном алгоритме - какая-то подозрительная вещь. Быть так хорошо не может. Спишем на случайность и очень недиверсифицированный корпус.

(Но потом я увеличила количество тестовых отзывов до 100 и accuracy была 0.77, это уже звучит хоть немножко правдоподобнее)

# Идеи к улучшению точности кода
1. Увеличение количества данных
2. Удаление Named Entities из трейн-данных (вряд ли упоминания имени режиссёра или имён главных персонажей характерны только для позитивных или только для негативных отзывов)
3. Отдельно рассматривать биграммы с отрицаниями (not + позитивно-окрашенное-слово - негативная окраска и наоборот).
4. Попытаться не убирать стоп-слова... (я это сделала, потому что мне показалось, что они не несут ничего полезного, но задумываясь об этом сейчас, в них есть ранее упомянутые отрицания)